In [ ]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily, Hourly
import warnings

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [ ]:
races = pd.read_csv(path+'data/races.csv')
results = pd.read_csv(path+'data/results.csv')

weather = races.iloc[:,[0,1,2,3,4,5,6]]
weather.shape

In [ ]:
### Just analyzing Spa 2021 as the weather was so bad the race was canelled.

s2021 = weather.query('season == 2021 & round == 12')

race = s2021.iloc[0]

race

In [ ]:
start = datetime(2018,1,1)
end = datetime(2018,1,28)
coord = Point(race.lat, race.long)

data = Daily(coord, start, end).fetch()

data.plot(y=['tavg'])
print(data.columns)
plt.show()

In [ ]:
def format_coco(coco):
    codes = {
        1.0 :'Clear',
        2.0 : 'Fair',
        3.0 : 'Cloudy',
        4.0 : 'Overcast',
        5.0 : 'Fog',
        6.0 : 'Freezing Fog',
        7.0 : 'Light Rain',
        8.0 : 'Rain',
        9.0 : 'Heavy Rain',
        10.0 : 'Freezing Rain',
        11.0 : 'Heavy Freezing Rain',
        12.0 : 'Sleet',
        13.0 : 'Heavy Sleet',
        14.0 : 'Light Snowfall',
        15.0 : 'Snowfall',
        16.0 : 'Heavy Snowfall',
        17.0 : 'Rain Shower',
        18.0 : 'Heavy Rain Shower',
        19.0 : 'Sleet Shower',
        20.0 : 'Heavy Sleet Shower',
        21.0 : 'Snow Shower',
        22.0 : 'Heavy Snow Shower',
        23.0 : 'Lightning',
        24.0 : 'Hail',
        25.0 : 'Thunderstorm',
        26.0 : 'Heavy Thunderstorm',
        27.0 : 'Storm',
    }
    return codes[coco]

In [ ]:
# start = datetime.strptime(race.date, '%Y-%m-%d')
# end = datetime.strptime(race.date, '%Y-%m-%d')
start = datetime(2021,1,1)
end = datetime(2021,12,31,23)
coord = Point(race.lat, race.long)

data = Hourly(coord, start, end).fetch()

weather = data.groupby('coco').rhum.describe().reset_index()
weather['coco'] = weather.coco.apply(lambda x: format_coco(x))

In [ ]:
results.head()

In [ ]:
weather.head()

In [ ]:
data.plot(y=['temp'])
plt.show()